# 01. Chains
chains in Langchain, elucidating their function and importance in the structure of the language model. We learn about the different types of chain and their use.

In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
import langchain as lc

import warnings
warnings.filterwarnings('ignore')

load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
template = """
Interprete the text and evaluate the text.
sentiment: is the text in a positive, neutral or negative sentiment?
subject: What subject is the text about? Use exactly one word.

Format the output as JSON with the following keys:
sentiment
subject

text: {input}
"""

llm = lc.OpenAI(temperature=0)
prompt_template = lc.PromptTemplate.from_template(template=template)
chain = lc.LLMChain(llm=llm, prompt=prompt_template)
chain.predict(input="I ordered Pizza Salami and it was awesome!")

### 1.1 Sequential Chains
Sometimes we want to pass the output from one model to a another model. This can be done with different SequentialChains.

In [ ]:
response_template = """
You are a helpful bot that creates a 'thank you' reponse text. 
If customers are unsatisfied, offer them a real world assitant to talk to. 
You will get a sentiment and subject as into and evaluate. 

text: {input}
"""
review_template = lc.PromptTemplate(input_variables=['input'], template=response_template)
review_chain = lc.LLMChain(llm=llm, prompt=review_template)
review_chain

In [ ]:
overall_chain = lc.chains.SimpleSequentialChain(chains=[chain, review_chain],
                                                verbose=True)
overall_chain.run(input="I ordered Pizza salami & was aweful!")


Chains can be more complex and not all sequential chains will be as simple as passing a single string as an argument and getting a single string as output for all steps in the chain

In [ ]:
# Let's create a more complex chain.

# This is an LLMChain to write a review given a dish name and the experience.
prompt_review = lc.PromptTemplate.from_template(
    template="You ordered {dish_name} and your experience was {experience}. Write a review: "
)
chain_review = lc.LLMChain(llm=llm, prompt=prompt_review, 
                           output_key="review")

# This is an LLMChain to write a follow-up comment given the restaurant review.
prompt_comment = lc.PromptTemplate.from_template(
    template="Given the restaurant review: {review}, write a follow-up comment: "
)
chain_comment = lc.LLMChain(llm=llm, prompt=prompt_comment, 
                            output_key="comment")

# This is an LLMChain to summarize a review.
prompt_summary = lc.PromptTemplate.from_template(
    template="Summarise the review in one short sentence: \n\n {review}"
)
chain_summary = lc.LLMChain(llm=llm, prompt=prompt_summary, 
                            output_key="summary")

# This is an LLMChain to translate a summary into Hindi.
prompt_translation = lc.PromptTemplate.from_template(
    template="Translate the summary to Hindi: \n\n {summary}"
)
chain_translation = lc.LLMChain(llm=llm, prompt=prompt_translation, 
                                output_key="hindi_translation"
)

overall_chain = lc.chains.SequentialChain(
    chains=[chain_review, chain_comment, chain_summary, chain_translation],
    input_variables=["dish_name", "experience"],
    output_variables=["review", "comment", "summary", "hindi_translation"],
)

output = overall_chain({"dish_name": "Pizza Salami", 
               "experience": "It was awful!"})

In [ ]:
def printOutput(d):
    for k,v in d.items():
        print(k,":",v)

printOutput(output)

Instead of chaining multiple chains together we can also use an LLM to decide which follow up chain is being used

In [ ]:
positive_template = """You are an AI bot that focuses on the positive side of things. \
Whenever you analyze a text, you look for the positive aspects and highlight them. \
Here is the text:
{input}"""

neutral_template = """You are an AI bot that has a neutral perspective. You just provide a balanced analysis of the text, \
not favoring any positive or negative aspects. Here is the text:
{input}"""

negative_template = """You are an AI bot that is designed to find the negative aspects in a text. \
You analyze a text and show the potential downsides. Here is the text:
{input}"""

In [ ]:
prompt_infos = [
    {
        "name": "positive",
        "description": "Good for analyzing positive sentiments",
        "prompt_template": positive_template,
    },
    {
        "name": "neutral",
        "description": "Good for analyzing neutral sentiments",
        "prompt_template": neutral_template,
    },
    {
        "name": "negative",
        "description": "Good for analyzing negative sentiments",
        "prompt_template": negative_template,
    },
]

destination_chains = {}
for p_info in prompt_infos:
    prompt = lc.PromptTemplate(template=p_info["prompt_template"], 
                            input_variables=["input"])
    chain = lc.LLMChain(llm=llm, prompt=prompt)
    destination_chains[p_info["name"]] = chain
destination_chains

In [ ]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = lc.chains.router.multi_prompt_prompt.MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = lc.PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=lc.chains.router.llm_router.RouterOutputParser(),
)

router_chain = lc.chains.router.llm_router.LLMRouterChain.from_llm(llm, router_prompt)

chain = lc.chains.router.MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=destination_chains["neutral"],
    verbose=True,
)

chain.run("I ordered Pizza Salami for 9.99$ and it was awesome!")

# -------------- END --------------

In [ ]:
import openai
from langchain import PromptTemplate, OpenAI, LLMChain
from langchain.llms import OpenAI
from langchain.chains import SequentialChain
from langchain.chains import SimpleSequentialChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE